<a href="https://colab.research.google.com/github/Jgallear/CSSP_brazil_23_24/blob/main/rescale_VHI_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install required non-standard libraries
!pip install rioxarray
!pip install rasterio
!pip install cartopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 78.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import glob
import rioxarray as riox
from rasterio.enums import Resampling
import xarray as xr
from rasterio import CRS

In [ ]:
# glob files together in list
fpath = "/content/drive/MyDrive/rs_data/VHI/VHI_monthly/"
VHI_flist = glob.glob(fpath + "/*.tif")
VHI_flist.sort()

In [ ]:
# define downscale factor
downscale_factor = 0.14399999999997704

In [ ]:
# loop through files in VHI file list
# create function to sort VHI list
def sort_glob_list(VHI_flist):
  year_list=[]
  month_list=[]
  m_strings=[]
  for i in range(len(VHI_flist)):
    VHI_f = VHI_flist[i] # each VHI file in list
    #print(VHI_f)
    #get year_month part of file string to use later
    year_monthi = VHI_f[-11:-4]
    #print(year_monthi)
    # sort files into correct order
    year_i = int(year_monthi[0:4])
    month_i = int(year_monthi[-2:])
    mstr = year_monthi[-2:]
    m_strings.append(mstr)
    #print(year_i)
    #print(month_i)
    year_list.append(year_i)
    month_list.append(month_i)
  # create dataframe to sort in order of month and year
  flist_df = pd.DataFrame({'fname':VHI_flist,'year':year_list,'month':month_list})
  flist_df.sort_values(by=['year','month'],ascending=True,inplace=True)
  print(flist_df)
  for index, row in flist_df.iterrows():
    print(row)
  return flist_df,m_strings

In [ ]:
flist_df,month_strings = sort_glob_list(VHI_flist)
fnames = flist_df['fname'].tolist()
year = flist_df['year'].tolist()
month = flist_df['month'].tolist()

                                                 fname  year  month
0    /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  1981     10
1    /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  1981     11
2    /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  1981     12
3    /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  1982      1
4    /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  1982      2
..                                                 ...   ...    ...
487  /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  2022      5
488  /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  2022      6
489  /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  2022      7
490  /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  2022      8
491  /content/drive/MyDrive/rs_data/VHI/VHI_monthly...  2022      9

[492 rows x 3 columns]
fname    /content/drive/MyDrive/rs_data/VHI/VHI_monthly...
year                                                  1981
month                                     

In [ ]:
print(len(month_strings))

In [ ]:
# loop through list of fnames and apply rioxarray functions to scale up to 0.25 degrees
for i in range(len(fnames)):
  fi = fnames[i]
  vhi_raster_i = riox.open_rasterio(fi,crs = CRS.from_epsg(3005))
  #specify new width and height
  new_width = vhi_raster_i.rio.width * downscale_factor
  new_height = vhi_raster_i.rio.height * downscale_factor
  # reproject raster as downsampled grid
  down_sampled_i = vhi_raster_i.rio.reproject(vhi_raster_i.rio.crs, shape=(int(new_height), int(new_width)), resampling=Resampling.average)
  #print(vhi_raster_i.rio.resolution(), vhi_raster_i.rio.resolution())
  # save output raster
  # /content/drive/MyDrive/rs_data/rescaled_VHI
  print('/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_'+str(year[i])+'_'+month_strings[i]+'.tif')
  down_sampled_i.rio.to_raster('/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_'+str(year[i])+'_'+month_strings[i]+'.tif')

/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1981_10.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1981_11.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1981_12.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_01.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_02.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_03.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_04.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_05.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_06.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_07.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_08.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_09.tif
/content/drive/MyDrive/rs_data/rescaled_VHI/rescaled_non_aligned_1982_10.tif

In [ ]:
# Rescaling now complete, need to use nearest neighbours in other script to generate interpolated grids